In [1]:
"""
NOTES: 
Requirements 
    -> Plotly Account to make charts (free version may work but max dataset length is limited)
    -> Quandl Account & API Key 
        -> Link to Quandl dataset https://www.quandl.com/data/CHRIS-Wiki-Continuous-Futures
"""

# imports (not all are used)
import pandas as pd
import numpy as np
import matplotlib
import cufflinks as cf
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import quandl
from datetime import datetime

# EuroDollar prices from quandl


In [2]:
# weekly Euro$ price data from '94 - November '18
xl = pd.read_excel('EuroDollarHistorical.xlsx', sheet_name='Sheet2', header=0, index_col='Date')
# first 11 contract maturities 
xl = xl.loc[:, 'ED1 Comdty':'ED11 Comdty']

In [3]:
# end date to be used
t = datetime.today().strftime('%Y-%m-%d')
# list containing specific ticker dataframes
ed = []
for x in range(1,len(xl.columns)+1):
    ticker = "CHRIS/CME_ED{x}".format(x=x)
    EDx = quandl.get(ticker, start_date='2018-11-17', end_date=t, collapse='weekly')['Last']
    ed.append(EDx)
# convert list of lists to dataframe
ED = pd.concat(ed, axis=1)
# match col names
ED.columns = xl.columns
# append final list
xl = xl.append(ED)

In [4]:
# convert contract prices to yields
yld = xl.apply(lambda x : (100-x)/100)

In [5]:
xl.iloc[-2,:]

ED1 Comdty     98.095
ED2 Comdty     98.280
ED3 Comdty     98.380
ED4 Comdty     98.460
ED5 Comdty     98.465
ED6 Comdty     98.540
ED7 Comdty     98.545
ED8 Comdty     98.545
ED9 Comdty     98.510
ED10 Comdty    98.510
ED11 Comdty    98.485
Name: 2019-12-01 00:00:00, dtype: float64

In [6]:
# calculate yield spread between consecutive maturities
sprd = yld.diff(axis=1)
# [sprd_chg] is the nominal weekly change in yield
sprd_chg = sprd.diff(axis=0)

In [7]:
sprd_chg[:2]

,ED1 Comdty,ED2 Comdty,ED3 Comdty,ED4 Comdty,ED5 Comdty,ED6 Comdty,ED7 Comdty,ED8 Comdty,ED9 Comdty,ED10 Comdty,ED11 Comdty
Date,,,,,,,,,,,
1994-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994-01-14,NaN,0.0004,0.0002,0.0002,0.0001,0.0001,0.0,-0.0001,-1.526557e-16,1.457168e-16,-0.0001


In [8]:
# create [lbl] to hold formatted col headers
lbl = []
for l in range(0, len(sprd_chg.columns)):
    if l < len(sprd_chg.columns)-1:
        row = sprd_chg.columns[l+1] + " - " + sprd_chg.columns[l]
        lbl.append(row)

In [9]:
# drop 1st column and row in [sprd_chg]
sprd_chg = sprd_chg.drop('ED1 Comdty', axis=1)
sprd_chg.columns = [lbl]
sprd_chg = sprd_chg[1:]
sprd_chg[:2]

,ED2 Comdty - ED1 Comdty,ED3 Comdty - ED2 Comdty,ED4 Comdty - ED3 Comdty,ED5 Comdty - ED4 Comdty,ED6 Comdty - ED5 Comdty,ED7 Comdty - ED6 Comdty,ED8 Comdty - ED7 Comdty,ED9 Comdty - ED8 Comdty,ED10 Comdty - ED9 Comdty,ED11 Comdty - ED10 Comdty
Date,,,,,,,,,,
1994-01-14,0.0004,2.000000e-04,0.0002,0.0001,0.0001,0.000000e+00,-0.0001,-1.526557e-16,1.457168e-16,-0.0001
1994-01-21,-0.0002,-1.387779e-16,-0.0003,-0.0001,0.0000,-6.938894e-18,-0.0001,1.457168e-16,0.000000e+00,0.0001


In [10]:
# drop 1st column in [sprd]
sprd = sprd.drop('ED1 Comdty', axis=1)
sprd.columns= [lbl]
sprd[:2]

,ED2 Comdty - ED1 Comdty,ED3 Comdty - ED2 Comdty,ED4 Comdty - ED3 Comdty,ED5 Comdty - ED4 Comdty,ED6 Comdty - ED5 Comdty,ED7 Comdty - ED6 Comdty,ED8 Comdty - ED7 Comdty,ED9 Comdty - ED8 Comdty,ED10 Comdty - ED9 Comdty,ED11 Comdty - ED10 Comdty
Date,,,,,,,,,,
1994-01-07,0.0027,0.0029,0.0039,0.0014,0.0023,0.002,0.0031,0.0005,0.0017,0.0014
1994-01-14,0.0031,0.0031,0.0041,0.0015,0.0024,0.002,0.0030,0.0005,0.0017,0.0013


In [11]:
# weekly average spread change
avgsc = sprd_chg.mean(axis=1)*10000
# weekly average spread
avgs = pd.DataFrame(sprd.mean(axis=1)*10000)
# combine prior 2 avgs into [sc_stats]
sc_stats = pd.concat([avgsc, avgs], axis=1)
sc_stats.columns = ['WeeklyChg', 'Avg']
# sprd_chg expressed in %
pct_chg = pd.DataFrame(sc_stats['Avg'].pct_change())
pct_chg.columns = ['PctChange']

In [12]:
# take the sum of all spreads and filter values where x<0
sums = pd.DataFrame(sprd.sum(axis=1), columns=['Sum'])
mask = (lambda x : x<0)
isInv = sprd[mask]

In [13]:
ED_pct = pd.DataFrame(xl.pct_change())

In [14]:
ED_pct[-2:]

,ED1 Comdty,ED2 Comdty,ED3 Comdty,ED4 Comdty,ED5 Comdty,ED6 Comdty,ED7 Comdty,ED8 Comdty,ED9 Comdty,ED10 Comdty,ED11 Comdty
Date,,,,,,,,,,,
2019-12-01,0.000127,-0.000153,-0.000152,-0.000152,-0.000152,-0.000152,-0.000152,-0.000203,-0.000203,-0.000203,-0.000305
2019-12-08,0.000178,0.000102,-0.000051,-0.000203,-0.000254,-0.000406,-0.000507,-0.000558,-0.000609,-0.000660,-0.000660


In [15]:
avgprc = pd.DataFrame(xl.mean(axis=1))
avgprc.columns = ['AvgPx']
avgprc['ChgPx'] = avgprc['AvgPx'].pct_change()
sigma = avgprc['AvgPx'].describe()[2]
roll = avgprc['AvgPx'].rolling(window=52, min_periods=2, axis=0).mean()

In [16]:
contangoSum = pd.DataFrame(isInv.sum(axis=1), columns=['contangoSum'])

In [17]:
trace0 = go.Scatter(
    name='Avg PX',
    x=sc_stats.index,
    y=avgprc.AvgPx,
    yaxis='y2'
    )
trace1 = go.Scatter(
    name='52 Week MA',
    x=sc_stats.index,
    y=roll,
    yaxis='y2'
    )
trace2 = go.Scatter(
    name='Avg Spread',
    x=sc_stats.index,
    y=sc_stats.Avg
    )
trace3 = go.Bar(
    name='PX %Chg',
    x=sc_stats.index,
    y=avgprc.ChgPx,
    yaxis='y3'
    )


layout = dict()
data=[trace0, trace1, trace2, trace3]
fig=dict(data=data, layout=layout)
fig['layout']['title'] = 'EuroDollar Consecutive Calendar Spreads'
fig['layout']['yaxis'] = dict(domain=[0.3,0.6], ticksuffix='bps', title='Average Spread')
fig['layout']['yaxis2'] = dict(domain=[0.6,1.0], tickprefix='$', title='Avg PX')
fig['layout']['yaxis3'] = dict(domain=[0.0,0.3], title='PX %Chg', tickformat='.02%')
fig['layout']['xaxis'] = dict(anchor='y3', title='Date')
fig['layout']['xaxis2'] = dict(overlaying='x', side='top', layer='above traces', range=[0.25,0])
fig['layout']['colorway'] = ['#2E91E5','#1CA71C','#FB0D0D','#DA16FF','#E15F99']
fig['layout']['font'] = dict(color='rgb(0,0,0)', family='Old Standard TT', size=13)
py.iplot(fig, filename='Average spreads vs Change')



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [18]:
contangoSum2 = contangoSum*10000

In [19]:
trace = go.Bar(
    x=sc_stats.index, 
    y=contangoSum2['contangoSum'],
    name='ContangoSum'
    )
layout = go.Layout(
    yaxis=dict(ticksuffix='bps'),
    title='CME EuroDollar Futures (mty 1:10) Cumulative Inversion',
    font=dict(color='rgb(0,0,0)', family='Old Standard TT', size=13)
    )
data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='ContangoSum')

In [20]:
isInv2=isInv*10000

In [21]:
trace = [go.Bar(
    x=sc_stats.index, 
    y=isInv2[col], 
    name=str(col)
)for col in isInv.columns]
data = trace
layout = go.Layout(
    barmode='stack',
    yaxis=dict(ticksuffix='bps'),
    title='CME EuroDollar Futures (mty 1:10) Inverted Contracts',
    font=dict(color='rgb(0,0,0)', family='Old Standard TT', size=13),
    legend=dict(orientation='h', font=dict(color='rgb(0,0,0)', family='Old Standard TT', size=10))
    )
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='InversionByContract')

In [22]:
trace0 = go.Scatter(
    name='Last',
    x=yld.columns,
    y=yld.iloc[-1].values,
    mode = 'lines+markers'
    )

trace1 = go.Scatter(
    name='1 Week',
    x=yld.columns,
    y=yld.iloc[-2].values,
    mode = 'lines+markers'
    )

trace2 = go.Scatter(
    name='1 Month',
    x=yld.columns,
    y=yld.iloc[-4].values,
    mode = 'lines+markers'
    )

trace3 = go.Scatter(
    name='1 Year',
    x=yld.columns,
    y=yld.iloc[-52].values,
    mode = 'lines+markers'
    )

trace4 = go.Scatter(
    name='10 Years',
    x=yld.columns,
    y=yld.iloc[-520].values,
    mode = 'lines+markers'
    )

layout = go.Layout(
    title='EuroDollar Curve',
    font=dict(color='rgb(0,0,0)', family='Old Standard TT', size=13),
    yaxis=dict(tickformat='.2%')

    )

data = [trace0,trace1,trace2,trace3,trace4]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='ED_Curve')